# 🎤 Common Voice Veri Seti - Apache Spark ile Preprocessing ve Eğitim

Bu notebook, Mozilla Common Voice veri setini indirip, Apache Spark ile preprocessing yaparak model eğitimi için hazırlar.

## İçindekiler:
1. Kütüphanelerin Kurulumu
2. Veri Setinin İndirilmesi
3. Spark Session Oluşturma
4. Veri Yükleme ve Keşif
5. Preprocessing Pipeline
6. Öznitelik Çıkarma (Feature Extraction)
7. Model Eğitimi
8. Değerlendirme

## 1. 📦 Kütüphanelerin Kurulumu

In [ ]:
# Gerekli kütüphaneleri yükle
!pip install pyspark
!pip install kaggle
!pip install librosa
!pip install soundfile
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install tqdm

In [ ]:
# Kütüphaneleri import et
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Spark kütüphaneleri
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, lit, udf, split, explode, lower, trim,
    regexp_replace, length, count, avg, max, min, sum,
    array, struct, concat, concat_ws
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    FloatType, ArrayType, DoubleType
)
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    StringIndexer, VectorAssembler, StandardScaler,
    MinMaxScaler, PCA, OneHotEncoder
)
from pyspark.ml.classification import (
    LogisticRegression, RandomForestClassifier,
    GBTClassifier, MultilayerPerceptronClassifier
)
from pyspark.ml.evaluation import (
    MulticlassClassificationEvaluator,
    BinaryClassificationEvaluator
)
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Ses işleme
import librosa
import soundfile as sf

print("✅ Tüm kütüphaneler başarıyla yüklendi!")

## 2. 📥 Common Voice Veri Setinin İndirilmesi

In [ ]:
# =============================================================================
# KAGGLE API AYARLARI
# =============================================================================
# Kaggle API anahtarınızı ayarlayın
# kaggle.json dosyanızı ~/.kaggle/ dizinine koyun veya aşağıdaki kodu kullanın

# Google Colab için:
# from google.colab import files
# files.upload()  # kaggle.json dosyasını yükleyin
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

DATASET_NAME = "mozillaorg/common-voice"
DATA_DIR = "/content/common-voice"

# Windows için alternatif path
# DATA_DIR = "C:/Users/info/Downloads/Veri Bilimi/common-voice"

print(f"📥 {DATASET_NAME} indiriliyor...")
print("⏳ Bu işlem birkaç dakika sürebilir...")

# Dizini oluştur
os.makedirs(DATA_DIR, exist_ok=True)

# Veri setini indir
!kaggle datasets download -d {DATASET_NAME} -p {DATA_DIR} --unzip

print(f"\n✅ Veri indirildi: {DATA_DIR}")
!ls -la {DATA_DIR}

In [ ]:
# İndirilen dosyaları kontrol et
import glob

def explore_directory(path):
    """Dizin yapısını keşfet"""
    print(f"\n📂 Dizin: {path}")
    print("=" * 50)
    
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:10]:  # İlk 10 dosyayı göster
            print(f"{subindent}📄 {file}")
        if len(files) > 10:
            print(f"{subindent}... ve {len(files) - 10} dosya daha")

explore_directory(DATA_DIR)

## 3. ⚡ Spark Session Oluşturma

In [ ]:
# =============================================================================
# SPARK SESSION OLUŞTURMA
# =============================================================================

def create_spark_session(app_name="CommonVoicePreprocessing", memory="4g"):
    """
    Spark session oluşturur
    
    Parameters:
    -----------
    app_name : str
        Uygulama adı
    memory : str
        Driver ve executor bellek miktarı
    
    Returns:
    --------
    SparkSession
    """
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.memory", memory) \
        .config("spark.executor.memory", memory) \
        .config("spark.sql.shuffle.partitions", "8") \
        .config("spark.default.parallelism", "8") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .getOrCreate()
    
    # Log seviyesini ayarla
    spark.sparkContext.setLogLevel("WARN")
    
    return spark

# Spark session başlat
spark = create_spark_session()

print("✅ Spark Session oluşturuldu!")
print(f"📌 Spark Version: {spark.version}")
print(f"📌 App Name: {spark.sparkContext.appName}")
print(f"📌 Master: {spark.sparkContext.master}")

## 4. 📊 Veri Yükleme ve Keşif

In [ ]:
# =============================================================================
# VERİ YÜKLEME
# =============================================================================

# Common Voice veri seti genellikle TSV formatında metadata içerir
# Dosya yapısına göre yolu güncelleyin

def load_metadata(spark, data_dir, file_pattern="*.tsv"):
    """
    Metadata dosyalarını yükle
    """
    # TSV dosyalarını bul
    tsv_files = glob.glob(os.path.join(data_dir, "**", file_pattern), recursive=True)
    
    if not tsv_files:
        # CSV dene
        tsv_files = glob.glob(os.path.join(data_dir, "**", "*.csv"), recursive=True)
    
    print(f"📄 Bulunan metadata dosyaları: {len(tsv_files)}")
    for f in tsv_files[:5]:
        print(f"   - {f}")
    
    return tsv_files

metadata_files = load_metadata(spark, DATA_DIR)

In [ ]:
# =============================================================================
# METADATA SPARK DATAFRAME OLARAK YÜKLE
# =============================================================================

def load_tsv_to_spark(spark, file_path, delimiter="\t"):
    """
    TSV/CSV dosyasını Spark DataFrame'e yükle
    """
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", delimiter) \
        .option("quote", '"') \
        .option("escape", '"') \
        .csv(file_path)
    
    return df

# İlk metadata dosyasını yükle (varsa)
if metadata_files:
    # TSV için tab, CSV için virgül
    delimiter = "\t" if metadata_files[0].endswith(".tsv") else ","
    df_metadata = load_tsv_to_spark(spark, metadata_files[0], delimiter)
    
    print("\n📊 Metadata Schema:")
    df_metadata.printSchema()
    
    print("\n📊 İlk 5 Satır:")
    df_metadata.show(5, truncate=False)
    
    print(f"\n📊 Toplam Kayıt Sayısı: {df_metadata.count():,}")
else:
    print("⚠️ Metadata dosyası bulunamadı. Lütfen DATA_DIR yolunu kontrol edin.")

In [ ]:
# =============================================================================
# VERİ KALİTESİ ANALİZİ
# =============================================================================

def analyze_data_quality(df):
    """
    Veri kalitesi analizi yapar
    """
    print("\n" + "=" * 60)
    print("📊 VERİ KALİTESİ ANALİZİ")
    print("=" * 60)
    
    # Toplam kayıt sayısı
    total_rows = df.count()
    print(f"\n📌 Toplam Kayıt: {total_rows:,}")
    
    # Sütun sayısı
    print(f"📌 Sütun Sayısı: {len(df.columns)}")
    
    # Null değer analizi
    print("\n📌 Null Değer Analizi:")
    null_counts = []
    for col_name in df.columns:
        null_count = df.filter(col(col_name).isNull()).count()
        null_pct = (null_count / total_rows) * 100
        null_counts.append((col_name, null_count, null_pct))
        if null_count > 0:
            print(f"   - {col_name}: {null_count:,} ({null_pct:.2f}%)")
    
    # Tekrar eden kayıtlar
    distinct_rows = df.distinct().count()
    duplicates = total_rows - distinct_rows
    print(f"\n📌 Tekrar Eden Kayıt: {duplicates:,}")
    
    return null_counts

if 'df_metadata' in dir():
    null_analysis = analyze_data_quality(df_metadata)

## 5. 🔧 Preprocessing Pipeline

In [ ]:
# =============================================================================
# METİN TEMİZLEME FONKSİYONLARI
# =============================================================================

def clean_text_column(df, column_name, new_column_name=None):
    """
    Metin sütununu temizler
    
    Parameters:
    -----------
    df : DataFrame
        Spark DataFrame
    column_name : str
        Temizlenecek sütun adı
    new_column_name : str, optional
        Yeni sütun adı
    
    Returns:
    --------
    DataFrame
    """
    if new_column_name is None:
        new_column_name = f"{column_name}_cleaned"
    
    df_cleaned = df.withColumn(
        new_column_name,
        # Küçük harfe çevir
        lower(
            # Özel karakterleri kaldır
            regexp_replace(
                # Fazla boşlukları kaldır
                regexp_replace(
                    # Baştaki ve sondaki boşlukları kaldır
                    trim(col(column_name)),
                    r"\s+", " "
                ),
                r"[^a-zA-ZğüşıöçĞÜŞİÖÇ0-9\s]", ""
            )
        )
    )
    
    return df_cleaned

print("✅ Metin temizleme fonksiyonları hazır!")

In [ ]:
# =============================================================================
# NULL DEĞER İŞLEME
# =============================================================================

def handle_null_values(df, strategy="drop", fill_value=None, subset=None):
    """
    Null değerleri işler
    
    Parameters:
    -----------
    df : DataFrame
        Spark DataFrame
    strategy : str
        'drop' - Null içeren satırları sil
        'fill' - Null değerleri doldur
        'mode' - En sık değerle doldur
        'mean' - Ortalama ile doldur (sayısal)
        'median' - Medyan ile doldur (sayısal)
    fill_value : any
        Doldurulacak değer (strategy='fill' için)
    subset : list
        İşlenecek sütunlar
    
    Returns:
    --------
    DataFrame
    """
    if strategy == "drop":
        return df.dropna(subset=subset)
    
    elif strategy == "fill":
        if fill_value is not None:
            return df.fillna(fill_value, subset=subset)
        else:
            # String için boş string, sayısal için 0
            fill_dict = {}
            for col_name, dtype in df.dtypes:
                if subset is None or col_name in subset:
                    if dtype == "string":
                        fill_dict[col_name] = ""
                    elif dtype in ["int", "bigint", "double", "float"]:
                        fill_dict[col_name] = 0
            return df.fillna(fill_dict)
    
    elif strategy == "mean":
        # Sayısal sütunlar için ortalama
        from pyspark.sql.functions import mean as spark_mean
        for col_name, dtype in df.dtypes:
            if subset is None or col_name in subset:
                if dtype in ["int", "bigint", "double", "float"]:
                    mean_val = df.select(spark_mean(col(col_name))).collect()[0][0]
                    if mean_val is not None:
                        df = df.fillna({col_name: mean_val})
        return df
    
    return df

print("✅ Null değer işleme fonksiyonu hazır!")

In [ ]:
# =============================================================================
# KATEGORİK VERİ DÖNÜŞÜMÜ
# =============================================================================

def encode_categorical_columns(df, columns, encoding_type="label"):
    """
    Kategorik sütunları encode eder
    
    Parameters:
    -----------
    df : DataFrame
        Spark DataFrame
    columns : list
        Encode edilecek sütunlar
    encoding_type : str
        'label' - Label Encoding
        'onehot' - One-Hot Encoding
    
    Returns:
    --------
    DataFrame, list of stages
    """
    stages = []
    
    for column in columns:
        if column not in df.columns:
            continue
            
        # String Indexer
        indexer = StringIndexer(
            inputCol=column,
            outputCol=f"{column}_index",
            handleInvalid="keep"
        )
        stages.append(indexer)
        
        if encoding_type == "onehot":
            # One-Hot Encoder
            encoder = OneHotEncoder(
                inputCols=[f"{column}_index"],
                outputCols=[f"{column}_onehot"]
            )
            stages.append(encoder)
    
    # Pipeline oluştur ve uygula
    if stages:
        pipeline = Pipeline(stages=stages)
        model = pipeline.fit(df)
        df_encoded = model.transform(df)
        return df_encoded, stages
    
    return df, stages

print("✅ Kategorik encoding fonksiyonu hazır!")

In [ ]:
# =============================================================================
# SAYISAL VERİ NORMALİZASYONU
# =============================================================================

def normalize_numerical_columns(df, columns, method="standard"):
    """
    Sayısal sütunları normalize eder
    
    Parameters:
    -----------
    df : DataFrame
        Spark DataFrame
    columns : list
        Normalize edilecek sütunlar
    method : str
        'standard' - StandardScaler (z-score)
        'minmax' - MinMaxScaler (0-1)
    
    Returns:
    --------
    DataFrame
    """
    # Önce VectorAssembler ile sütunları birleştir
    assembler = VectorAssembler(
        inputCols=columns,
        outputCol="features_raw",
        handleInvalid="skip"
    )
    df_assembled = assembler.transform(df)
    
    # Scaler seç
    if method == "standard":
        scaler = StandardScaler(
            inputCol="features_raw",
            outputCol="features_scaled",
            withStd=True,
            withMean=True
        )
    else:  # minmax
        scaler = MinMaxScaler(
            inputCol="features_raw",
            outputCol="features_scaled"
        )
    
    # Scaler'ı uygula
    scaler_model = scaler.fit(df_assembled)
    df_scaled = scaler_model.transform(df_assembled)
    
    return df_scaled

print("✅ Normalizasyon fonksiyonu hazır!")

## 6. 🎵 Ses Öznitelik Çıkarma (Audio Feature Extraction)

In [ ]:
# =============================================================================
# SES ÖZNİTELİK ÇIKARMA FONKSİYONLARI
# =============================================================================

def extract_audio_features(audio_path, sr=16000, n_mfcc=13):
    """
    Ses dosyasından öznitelik çıkarır
    
    Parameters:
    -----------
    audio_path : str
        Ses dosyası yolu
    sr : int
        Örnekleme oranı
    n_mfcc : int
        MFCC sayısı
    
    Returns:
    --------
    dict : Öznitelikler
    """
    try:
        # Ses dosyasını yükle
        y, sr = librosa.load(audio_path, sr=sr)
        
        # Temel öznitelikler
        features = {}
        
        # 1. MFCC (Mel-Frequency Cepstral Coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        for i in range(n_mfcc):
            features[f'mfcc_{i}_mean'] = np.mean(mfccs[i])
            features[f'mfcc_{i}_std'] = np.std(mfccs[i])
        
        # 2. Chroma Features
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        features['chroma_mean'] = np.mean(chroma)
        features['chroma_std'] = np.std(chroma)
        
        # 3. Spectral Centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        features['spectral_centroid_mean'] = np.mean(spectral_centroid)
        features['spectral_centroid_std'] = np.std(spectral_centroid)
        
        # 4. Spectral Bandwidth
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        features['spectral_bandwidth_mean'] = np.mean(spectral_bandwidth)
        features['spectral_bandwidth_std'] = np.std(spectral_bandwidth)
        
        # 5. Spectral Rolloff
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        features['spectral_rolloff_mean'] = np.mean(spectral_rolloff)
        features['spectral_rolloff_std'] = np.std(spectral_rolloff)
        
        # 6. Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(y)
        features['zcr_mean'] = np.mean(zcr)
        features['zcr_std'] = np.std(zcr)
        
        # 7. RMS Energy
        rms = librosa.feature.rms(y=y)
        features['rms_mean'] = np.mean(rms)
        features['rms_std'] = np.std(rms)
        
        # 8. Duration
        features['duration'] = librosa.get_duration(y=y, sr=sr)
        
        # 9. Tempo
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        features['tempo'] = tempo
        
        return features
        
    except Exception as e:
        print(f"⚠️ Hata: {audio_path} - {str(e)}")
        return None

print("✅ Ses öznitelik çıkarma fonksiyonu hazır!")

In [ ]:
# =============================================================================
# TOPLU ÖZNİTELİK ÇIKARMA
# =============================================================================

def extract_features_batch(audio_paths, sr=16000, n_mfcc=13):
    """
    Birden fazla ses dosyasından öznitelik çıkarır
    
    Parameters:
    -----------
    audio_paths : list
        Ses dosyası yolları listesi
    sr : int
        Örnekleme oranı
    n_mfcc : int
        MFCC sayısı
    
    Returns:
    --------
    pd.DataFrame : Öznitelik dataframe'i
    """
    all_features = []
    
    for audio_path in tqdm(audio_paths, desc="Öznitelik çıkarılıyor"):
        features = extract_audio_features(audio_path, sr, n_mfcc)
        if features is not None:
            features['file_path'] = audio_path
            all_features.append(features)
    
    df_features = pd.DataFrame(all_features)
    return df_features

print("✅ Toplu öznitelik çıkarma fonksiyonu hazır!")

In [ ]:
# =============================================================================
# SES DOSYALARINI BUL VE ÖZNİTELİK ÇIKAR
# =============================================================================

# Ses dosyalarını bul
audio_extensions = ['*.mp3', '*.wav', '*.flac', '*.ogg', '*.m4a']
audio_files = []

for ext in audio_extensions:
    audio_files.extend(glob.glob(os.path.join(DATA_DIR, "**", ext), recursive=True))

print(f"📊 Bulunan ses dosyası sayısı: {len(audio_files)}")

if audio_files:
    # Örnek olarak ilk 100 dosyadan öznitelik çıkar (tam veri seti için sınırı kaldırın)
    sample_size = min(100, len(audio_files))
    print(f"\n🔧 İlk {sample_size} dosyadan öznitelik çıkarılıyor...")
    
    df_audio_features = extract_features_batch(audio_files[:sample_size])
    
    print(f"\n✅ {len(df_audio_features)} dosyadan öznitelik çıkarıldı!")
    print(f"📊 Öznitelik sayısı: {len(df_audio_features.columns)}")
    
    # İlk birkaç satırı göster
    print("\n📊 Öznitelik Önizlemesi:")
    display(df_audio_features.head())
else:
    print("⚠️ Ses dosyası bulunamadı. Lütfen DATA_DIR yolunu kontrol edin.")

In [ ]:
# =============================================================================
# ÖZNİTELİKLERİ SPARK DATAFRAME'E DÖNÜŞTÜR
# =============================================================================

if 'df_audio_features' in dir() and len(df_audio_features) > 0:
    # Pandas DataFrame'i Spark DataFrame'e dönüştür
    spark_audio_features = spark.createDataFrame(df_audio_features)
    
    print("\n📊 Spark Audio Features Schema:")
    spark_audio_features.printSchema()
    
    print("\n📊 İstatistikler:")
    spark_audio_features.describe().show()

## 7. 🤖 Model Eğitimi

In [ ]:
# =============================================================================
# EĞİTİM VERİSİ HAZIRLA
# =============================================================================

def prepare_training_data(df, feature_cols, label_col, test_size=0.2):
    """
    Eğitim verisini hazırlar
    
    Parameters:
    -----------
    df : DataFrame
        Spark DataFrame
    feature_cols : list
        Öznitelik sütunları
    label_col : str
        Etiket sütunu
    test_size : float
        Test seti oranı
    
    Returns:
    --------
    train_df, test_df, pipeline
    """
    stages = []
    
    # 1. Label Indexer
    label_indexer = StringIndexer(
        inputCol=label_col,
        outputCol="label",
        handleInvalid="keep"
    )
    stages.append(label_indexer)
    
    # 2. Vector Assembler
    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features_raw",
        handleInvalid="skip"
    )
    stages.append(assembler)
    
    # 3. Scaler
    scaler = StandardScaler(
        inputCol="features_raw",
        outputCol="features",
        withStd=True,
        withMean=True
    )
    stages.append(scaler)
    
    # Pipeline oluştur
    pipeline = Pipeline(stages=stages)
    pipeline_model = pipeline.fit(df)
    df_prepared = pipeline_model.transform(df)
    
    # Train/Test split
    train_df, test_df = df_prepared.randomSplit([1-test_size, test_size], seed=42)
    
    print(f"✅ Eğitim seti: {train_df.count():,} kayıt")
    print(f"✅ Test seti: {test_df.count():,} kayıt")
    
    return train_df, test_df, pipeline_model

print("✅ Eğitim verisi hazırlama fonksiyonu hazır!")

In [ ]:
# =============================================================================
# MODEL EĞİTİMİ
# =============================================================================

def train_models(train_df, test_df):
    """
    Birden fazla model eğitir ve karşılaştırır
    
    Parameters:
    -----------
    train_df : DataFrame
        Eğitim verisi
    test_df : DataFrame
        Test verisi
    
    Returns:
    --------
    dict : Model sonuçları
    """
    results = {}
    
    # Evaluator
    evaluator = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="accuracy"
    )
    
    # 1. Logistic Regression
    print("\n🔄 Logistic Regression eğitiliyor...")
    lr = LogisticRegression(
        featuresCol="features",
        labelCol="label",
        maxIter=100,
        regParam=0.01
    )
    lr_model = lr.fit(train_df)
    lr_predictions = lr_model.transform(test_df)
    lr_accuracy = evaluator.evaluate(lr_predictions)
    results['Logistic Regression'] = {'model': lr_model, 'accuracy': lr_accuracy}
    print(f"   ✅ Accuracy: {lr_accuracy:.4f}")
    
    # 2. Random Forest
    print("\n🔄 Random Forest eğitiliyor...")
    rf = RandomForestClassifier(
        featuresCol="features",
        labelCol="label",
        numTrees=100,
        maxDepth=10,
        seed=42
    )
    rf_model = rf.fit(train_df)
    rf_predictions = rf_model.transform(test_df)
    rf_accuracy = evaluator.evaluate(rf_predictions)
    results['Random Forest'] = {'model': rf_model, 'accuracy': rf_accuracy}
    print(f"   ✅ Accuracy: {rf_accuracy:.4f}")
    
    # 3. Gradient Boosted Trees
    print("\n🔄 Gradient Boosted Trees eğitiliyor...")
    # GBT sadece binary classification destekler, multi-class için RF kullanılabilir
    # Burada label sayısını kontrol edelim
    num_labels = train_df.select("label").distinct().count()
    
    if num_labels == 2:
        gbt = GBTClassifier(
            featuresCol="features",
            labelCol="label",
            maxIter=50,
            maxDepth=5,
            seed=42
        )
        gbt_model = gbt.fit(train_df)
        gbt_predictions = gbt_model.transform(test_df)
        gbt_accuracy = evaluator.evaluate(gbt_predictions)
        results['GBT'] = {'model': gbt_model, 'accuracy': gbt_accuracy}
        print(f"   ✅ Accuracy: {gbt_accuracy:.4f}")
    else:
        print(f"   ⚠️ GBT multi-class desteklemez ({num_labels} sınıf var)")
    
    # 4. Multilayer Perceptron
    print("\n🔄 Multilayer Perceptron eğitiliyor...")
    # Öznitelik boyutunu al
    feature_size = len(train_df.select("features").first()[0])
    
    layers = [feature_size, 128, 64, num_labels]
    
    mlp = MultilayerPerceptronClassifier(
        featuresCol="features",
        labelCol="label",
        layers=layers,
        maxIter=100,
        seed=42
    )
    mlp_model = mlp.fit(train_df)
    mlp_predictions = mlp_model.transform(test_df)
    mlp_accuracy = evaluator.evaluate(mlp_predictions)
    results['MLP'] = {'model': mlp_model, 'accuracy': mlp_accuracy}
    print(f"   ✅ Accuracy: {mlp_accuracy:.4f}")
    
    return results

print("✅ Model eğitim fonksiyonu hazır!")

In [ ]:
# =============================================================================
# CROSS VALIDATION İLE MODEL OPTİMİZASYONU
# =============================================================================

def optimize_model_with_cv(train_df, model_type="rf", num_folds=5):
    """
    Cross-validation ile model optimizasyonu
    
    Parameters:
    -----------
    train_df : DataFrame
        Eğitim verisi
    model_type : str
        'rf' - Random Forest
        'lr' - Logistic Regression
    num_folds : int
        Fold sayısı
    
    Returns:
    --------
    best_model, cv_model
    """
    print(f"\n🔄 {model_type.upper()} için Cross-Validation başlıyor...")
    
    if model_type == "rf":
        model = RandomForestClassifier(
            featuresCol="features",
            labelCol="label",
            seed=42
        )
        paramGrid = ParamGridBuilder() \
            .addGrid(model.numTrees, [50, 100, 200]) \
            .addGrid(model.maxDepth, [5, 10, 15]) \
            .build()
    else:  # lr
        model = LogisticRegression(
            featuresCol="features",
            labelCol="label"
        )
        paramGrid = ParamGridBuilder() \
            .addGrid(model.regParam, [0.01, 0.1, 0.5]) \
            .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0]) \
            .build()
    
    evaluator = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="accuracy"
    )
    
    crossval = CrossValidator(
        estimator=model,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        numFolds=num_folds,
        seed=42
    )
    
    cv_model = crossval.fit(train_df)
    
    print(f"✅ En iyi CV skoru: {max(cv_model.avgMetrics):.4f}")
    
    return cv_model.bestModel, cv_model

print("✅ Cross-validation fonksiyonu hazır!")

## 8. 📈 Model Değerlendirme

In [ ]:
# =============================================================================
# DEĞERLENDİRME FONKSİYONLARI
# =============================================================================

def evaluate_model(predictions, label_col="label", prediction_col="prediction"):
    """
    Model performansını değerlendirir
    
    Parameters:
    -----------
    predictions : DataFrame
        Tahmin sonuçları
    label_col : str
        Gerçek etiket sütunu
    prediction_col : str
        Tahmin sütunu
    
    Returns:
    --------
    dict : Metrikler
    """
    metrics = {}
    
    # Multi-class evaluator
    evaluator = MulticlassClassificationEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col
    )
    
    # Accuracy
    metrics['accuracy'] = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
    
    # Precision
    metrics['precision'] = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    
    # Recall
    metrics['recall'] = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
    
    # F1 Score
    metrics['f1'] = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
    
    print("\n" + "=" * 50)
    print("📊 MODEL DEĞERLENDİRME SONUÇLARI")
    print("=" * 50)
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1 Score:  {metrics['f1']:.4f}")
    
    return metrics

print("✅ Değerlendirme fonksiyonu hazır!")

In [ ]:
# =============================================================================
# CONFUSION MATRIX GÖRSELLEŞTİRME
# =============================================================================

def plot_confusion_matrix(predictions, label_col="label", prediction_col="prediction"):
    """
    Confusion matrix görselleştirir
    """
    # Pandas'a dönüştür
    pred_df = predictions.select(label_col, prediction_col).toPandas()
    
    # Confusion matrix hesapla
    from sklearn.metrics import confusion_matrix, classification_report
    
    cm = confusion_matrix(pred_df[label_col], pred_df[prediction_col])
    
    # Görselleştir
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Gerçek Değer')
    plt.xlabel('Tahmin')
    plt.tight_layout()
    plt.show()
    
    # Classification report
    print("\n📊 Classification Report:")
    print(classification_report(pred_df[label_col], pred_df[prediction_col]))

print("✅ Confusion matrix fonksiyonu hazır!")

In [ ]:
# =============================================================================
# MODEL SONUÇLARINI KARŞILAŞTIR
# =============================================================================

def compare_models(results):
    """
    Model sonuçlarını karşılaştırır ve görselleştirir
    """
    model_names = list(results.keys())
    accuracies = [results[m]['accuracy'] for m in model_names]
    
    # Görselleştir
    plt.figure(figsize=(10, 6))
    bars = plt.bar(model_names, accuracies, color=['#3498db', '#2ecc71', '#e74c3c', '#9b59b6'])
    
    # Değerleri bar üzerine yaz
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{acc:.4f}', ha='center', va='bottom', fontsize=12)
    
    plt.title('Model Karşılaştırması', fontsize=14)
    plt.xlabel('Model', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.ylim(0, 1.1)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # En iyi model
    best_model_name = max(results, key=lambda x: results[x]['accuracy'])
    print(f"\n🏆 En iyi model: {best_model_name} (Accuracy: {results[best_model_name]['accuracy']:.4f})")

print("✅ Model karşılaştırma fonksiyonu hazır!")

## 9. 💾 Model Kaydetme ve Yükleme

In [ ]:
# =============================================================================
# MODEL KAYDETME
# =============================================================================

def save_model(model, path):
    """
    Modeli kaydeder
    """
    model.write().overwrite().save(path)
    print(f"✅ Model kaydedildi: {path}")

def load_model(model_class, path):
    """
    Modeli yükler
    """
    from pyspark.ml.classification import (
        RandomForestClassificationModel,
        LogisticRegressionModel,
        MultilayerPerceptronClassificationModel
    )
    
    model_classes = {
        'rf': RandomForestClassificationModel,
        'lr': LogisticRegressionModel,
        'mlp': MultilayerPerceptronClassificationModel
    }
    
    model = model_classes[model_class].load(path)
    print(f"✅ Model yüklendi: {path}")
    return model

print("✅ Model kaydetme/yükleme fonksiyonları hazır!")

## 10. 🚀 Tam Pipeline Çalıştırma

In [ ]:
# =============================================================================
# TAM PIPELINE
# =============================================================================

def run_full_pipeline(data_dir, label_column=None):
    """
    Tam pipeline çalıştırır
    
    Parameters:
    -----------
    data_dir : str
        Veri dizini
    label_column : str
        Etiket sütunu adı (None ise otomatik tespit edilir)
    """
    print("\n" + "=" * 60)
    print("🚀 TAM PIPELINE BAŞLATILIYOR")
    print("=" * 60)
    
    # 1. Ses dosyalarını bul
    print("\n📂 1. Ses dosyaları aranıyor...")
    audio_extensions = ['*.mp3', '*.wav', '*.flac', '*.ogg', '*.m4a']
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(glob.glob(os.path.join(data_dir, "**", ext), recursive=True))
    print(f"   ✅ {len(audio_files)} ses dosyası bulundu")
    
    if not audio_files:
        print("   ⚠️ Ses dosyası bulunamadı!")
        return None
    
    # 2. Öznitelik çıkar
    print("\n🔧 2. Öznitelikler çıkarılıyor...")
    df_features = extract_features_batch(audio_files)
    print(f"   ✅ {len(df_features)} dosyadan öznitelik çıkarıldı")
    
    # 3. Spark DataFrame'e dönüştür
    print("\n⚡ 3. Spark DataFrame oluşturuluyor...")
    spark_df = spark.createDataFrame(df_features)
    print(f"   ✅ DataFrame oluşturuldu")
    
    # 4. Eğer etiket varsa model eğit
    if label_column and label_column in spark_df.columns:
        print(f"\n🤖 4. Model eğitimi başlıyor (Etiket: {label_column})...")
        
        # Öznitelik sütunlarını al (etiket ve file_path hariç)
        feature_cols = [c for c in spark_df.columns if c not in [label_column, 'file_path']]
        
        # Eğitim verisini hazırla
        train_df, test_df, prep_pipeline = prepare_training_data(
            spark_df, feature_cols, label_column
        )
        
        # Model eğit
        results = train_models(train_df, test_df)
        
        # Karşılaştır
        compare_models(results)
        
        return results
    else:
        print("\n⚠️ Etiket sütunu bulunamadı. Sadece öznitelik çıkarma yapıldı.")
        return spark_df

print("✅ Tam pipeline fonksiyonu hazır!")
print("\n💡 Kullanım: results = run_full_pipeline(DATA_DIR, label_column='gender')")

In [ ]:
# =============================================================================
# PIPELINE'I ÇALIŞTIR
# =============================================================================

# Etiket sütunu varsa belirtin (örn: 'gender', 'age', 'accent' vb.)
# Common Voice veri setinde genellikle bu etiketler bulunur

# Örnek kullanım:
# results = run_full_pipeline(DATA_DIR, label_column='gender')

# Etiket olmadan sadece öznitelik çıkarma:
# df_features = run_full_pipeline(DATA_DIR)

## 11. 🧹 Temizlik

In [ ]:
# =============================================================================
# SPARK SESSION'I KAPAT
# =============================================================================

# İşlem bittiğinde Spark session'ı kapat
# spark.stop()
# print("✅ Spark session kapatıldı")

---

## 📝 Notlar

### Veri Seti Hakkında
- Mozilla Common Voice, çeşitli dillerde ses kayıtları içeren açık kaynaklı bir veri setidir
- Her kayıt için metadata (cinsiyet, yaş, aksan vb.) bulunabilir

### Preprocessing Adımları
1. **Veri İndirme**: Kaggle API ile
2. **Ses Öznitelik Çıkarma**: MFCC, Chroma, Spectral öznitelikleri
3. **Veri Temizleme**: Null değerler, normalizasyon
4. **Model Eğitimi**: Logistic Regression, Random Forest, MLP

### İpuçları
- Büyük veri setleri için `sample_size` parametresini artırın
- Bellek sorunları için Spark config ayarlarını düzenleyin
- Google Colab'da çalışırken disk alanına dikkat edin